<a href="https://colab.research.google.com/github/financieras/big_data/blob/main/python/groupby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guía Completa de `groupby` en Pandas

Este notebook proporciona una explicación detallada y progresiva del uso de `groupby` en Pandas, desde los conceptos fundamentales hasta técnicas avanzadas.

## 📚 Teoría: ¿Qué es `groupby`?

### El paradigma Split-Apply-Combine

`groupby` es una de las herramientas más poderosas de Pandas. Implementa el paradigma **Split-Apply-Combine**:

1. **Split (Dividir)**: Separa los datos en grupos basándose en algún criterio (valores de una o más columnas)
2. **Apply (Aplicar)**: Aplica una función a cada grupo de forma independiente
3. **Combine (Combinar)**: Combina los resultados en una estructura de datos

### ¿Cuándo usar `groupby`?

- Cuando necesitas **agregar datos** por categorías (suma, media, conteo, etc.)
- Para calcular **estadísticas por grupos**
- Cuando quieres **transformar datos** dentro de cada grupo (normalización, ranking, etc.)
- Para **filtrar grupos completos** basándote en alguna condición

### Sintaxis básica

```python
df.groupby('columna')['otra_columna'].funcion_agregacion()
```

o para múltiples columnas:

```python
df.groupby(['col1', 'col2']).agg({'col3': 'sum', 'col4': 'mean'})
```

---
## 🎯 Ejemplo 1: Agregación básica por columna

**Objetivo**: Entender el concepto básico de agrupar datos y calcular agregaciones simples.

Este ejemplo crea un DataFrame simple, agrupa los datos por una columna y calcula la suma de otra columna para cada grupo.

In [1]:
import pandas as pd
import numpy as np

# 1. Crear un DataFrame de ejemplo
data = {
    'Categoría': ['A', 'B', 'A', 'C', 'B', 'C', 'A'],
    'Valor': [10, 15, 20, 5, 25, 30, 10]
}
df = pd.DataFrame(data)

print("--- DataFrame Original ---")
print(f"\nForma: {df.shape}\n")
df

--- DataFrame Original ---

Forma: (7, 2)



,Categoría,Valor
0,A,10
1,B,15
2,A,20
3,C,5
4,B,25
5,C,30
6,A,10


In [4]:
# 2. Usar groupby para agrupar por 'Categoría' y calcular la suma de 'Valor'
df_suma = df.groupby('Categoría')['Valor'].sum()

print(f"Tipo de resultado: {type(df_suma)}\n")
print("\n--- Resultado: Suma de 'Valor' por 'Categoría' ---\n")
df_suma

Tipo de resultado: <class 'pandas.core.series.Series'>


--- Resultado: Suma de 'Valor' por 'Categoría' ---



,Valor
Categoría,
A,40
B,40
C,35


In [6]:
# 3. Calcular múltiples estadísticas a la vez
df_stats = df.groupby('Categoría')['Valor'].agg(['sum', 'mean', 'count', 'min', 'max'])

print("\n--- Múltiples estadísticas por 'Categoría' ---\n")
df_stats


--- Múltiples estadísticas por 'Categoría' ---



,sum,mean,count,min,max
Categoría,,,,,
A,40,13.333333,3,10,20
B,40,20.000000,2,15,25
C,35,17.500000,2,5,30


### 💡 Explicación del Código

1. **`df.groupby('Categoría')`**: Divide el DataFrame en 3 grupos (A, B, C)
2. **`['Valor']`**: Selecciona la columna sobre la que aplicaremos operaciones
3. **`.sum()`**: Aplica la suma a cada grupo
4. **`.agg([...])`**: Permite aplicar múltiples funciones de agregación simultáneamente

**Nota importante**: El resultado es una **Serie** cuando aplicamos una función, o un **DataFrame** cuando usamos `agg()` con múltiples funciones.

---
## 🎯 Ejemplo 2: Agrupación por múltiples columnas

**Objetivo**: Aprender a agrupar por más de una columna y aplicar diferentes funciones a diferentes columnas.

Esto es útil cuando queremos analizar datos con múltiples dimensiones (ej: ventas por ciudad y año).

In [7]:
import pandas as pd

# Crear un DataFrame más realista
data = {
    'Ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Barcelona', 'Madrid'],
    'Año': [2020, 2020, 2021, 2021, 2020, 2021],
    'Ventas': [100, 150, 200, 50, 120, 180],
    'Ingresos': [1000, 1500, 2000, 500, 1200, 1800]
}
df = pd.DataFrame(data)

print("--- DataFrame Original ---\n")
df

--- DataFrame Original ---



,Ciudad,Año,Ventas,Ingresos
0,Madrid,2020,100,1000
1,Barcelona,2020,150,1500
2,Madrid,2021,200,2000
3,Valencia,2021,50,500
4,Barcelona,2020,120,1200
5,Madrid,2021,180,1800


In [8]:
# Agrupar por 'Ciudad' y 'Año', aplicar diferentes funciones a cada columna
df_agrupado = df.groupby(['Ciudad', 'Año']).agg({
    'Ventas': ['sum', 'mean'],
    'Ingresos': 'sum'
})

print("\n--- Agrupación por Ciudad y Año ---\n")
df_agrupado


--- Agrupación por Ciudad y Año ---



Ventas        Ingresos
                  sum   mean      sum
Ciudad    Año                        
Barcelona 2020    270  135.0     2700
Madrid    2020    100  100.0     1000
          2021    380  190.0     3800
Valencia  2021     50   50.0      500

In [9]:
# Resetear el índice para obtener un DataFrame más manejable
df_plano = df.groupby(['Ciudad', 'Año']).agg({
    'Ventas': 'sum',
    'Ingresos': 'sum'
}).reset_index()

print("\n--- DataFrame con índice reseteado ---\n")
df_plano


--- DataFrame con índice reseteado ---



,Ciudad,Año,Ventas,Ingresos
0,Barcelona,2020,270,2700
1,Madrid,2020,100,1000
2,Madrid,2021,380,3800
3,Valencia,2021,50,500


### 💡 Explicación

- **Multi-índice**: Al agrupar por múltiples columnas, obtenemos un índice jerárquico (MultiIndex)
- **`reset_index()`**: Convierte el índice jerárquico en columnas normales, útil para visualización o exportación
- **Diccionario en `agg()`**: Permite aplicar diferentes funciones a diferentes columnas

---
## 🎯 Ejemplo 3: Diferencia entre `size()`, `count()` y `len()`

**Objetivo**: Entender las diferencias entre las funciones de conteo en groupby.

Esta es una fuente común de confusión para principiantes.

In [10]:
import pandas as pd
import numpy as np

# DataFrame con algunos valores NaN (nulos)
data = {
    'Grupo': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Valor1': [10, 20, np.nan, 40, 50, 60],
    'Valor2': [100, np.nan, 300, 400, np.nan, 600]
}
df = pd.DataFrame(data)

print("--- DataFrame con valores NaN ---\n")
df

--- DataFrame con valores NaN ---



,Grupo,Valor1,Valor2
0,A,10.0,100.0
1,A,20.0,NaN
2,B,NaN,300.0
3,B,40.0,400.0
4,C,50.0,NaN
5,C,60.0,600.0


In [11]:
# size() cuenta TODAS las filas de cada grupo (incluye NaN)
print("\n--- size() - Cuenta todas las filas por grupo ---\n")
df.groupby('Grupo').size()


--- size() - Cuenta todas las filas por grupo ---



,0
Grupo,
A,2
B,2
C,2


In [12]:
# count() cuenta valores NO NULOS por columna
print("\n--- count() - Cuenta valores no nulos por columna ---\n")
df.groupby('Grupo').count()


--- count() - Cuenta valores no nulos por columna ---



,Valor1,Valor2
Grupo,,
A,2,1
B,1,2
C,2,1


In [13]:
# Comparación directa
resumen = pd.DataFrame({
    'Total_filas': df.groupby('Grupo').size(),
    'Valor1_no_nulos': df.groupby('Grupo')['Valor1'].count(),
    'Valor2_no_nulos': df.groupby('Grupo')['Valor2'].count()
})

print("\n--- Comparación clara ---\n")
resumen


--- Comparación clara ---



,Total_filas,Valor1_no_nulos,Valor2_no_nulos
Grupo,,,
A,2,2,1
B,2,1,2
C,2,2,1


### 💡 Explicación

- **`size()`**: Cuenta el número total de filas en cada grupo (incluye NaN)
- **`count()`**: Cuenta valores no nulos en cada columna
- **`len()`**: Similar a `size()` cuando se usa con `apply()`

**Regla práctica**: Usa `size()` para contar filas, `count()` para contar valores válidos.

---
## 🎯 Ejemplo 4: Transformación con `transform()`

**Objetivo**: Usar `transform()` para modificar valores manteniendo la forma original del DataFrame.

`transform()` es diferente de `agg()` porque devuelve un resultado con el mismo tamaño que el grupo original.

In [14]:
import pandas as pd

# Crear un DataFrame de ejemplo
data = {
    'Equipo': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Puntuación': [10, 20, 15, 25, 30, 10]
}
df = pd.DataFrame(data)

print("--- DataFrame Original ---\n")
df

--- DataFrame Original ---



,Equipo,Puntuación
0,A,10
1,A,20
2,B,15
3,B,25
4,C,30
5,C,10


In [15]:
# 1. Normalizar puntuaciones dentro de cada equipo (0-1)
def normalizar(grupo):
    return (grupo - grupo.min()) / (grupo.max() - grupo.min())

df['Puntuación_normalizada'] = df.groupby('Equipo')['Puntuación'].transform(normalizar)

# 2. Calcular desviación respecto a la media del equipo
df['Desviación_media'] = df.groupby('Equipo')['Puntuación'].transform(
    lambda x: x - x.mean()
)

# 3. Añadir la media del equipo como nueva columna
df['Media_equipo'] = df.groupby('Equipo')['Puntuación'].transform('mean')

print("\n--- DataFrame con transformaciones ---\n")
df


--- DataFrame con transformaciones ---



,Equipo,Puntuación,Puntuación_normalizada,Desviación_media,Media_equipo
0,A,10,0.0,-5.0,15.0
1,A,20,1.0,5.0,15.0
2,B,15,0.0,-5.0,20.0
3,B,25,1.0,5.0,20.0
4,C,30,1.0,10.0,20.0
5,C,10,0.0,-10.0,20.0


### 💡 Explicación

- **`transform()`** mantiene el índice original del DataFrame
- Es perfecto para **añadir columnas calculadas** basadas en estadísticas del grupo
- Puedes usar funciones predefinidas (`'mean'`, `'sum'`) o funciones personalizadas

**Diferencia clave**:
- `agg()` → Reduce grupos (devuelve menos filas)
- `transform()` → Mantiene el tamaño (devuelve igual número de filas)

---
## 🎯 Ejemplo 5: Filtrado de grupos con `filter()`

**Objetivo**: Seleccionar grupos completos que cumplan cierta condición.

`filter()` es útil cuando queremos mantener o descartar grupos enteros basándonos en algún criterio.

In [16]:
import pandas as pd

# Crear un DataFrame de ejemplo
data = {
    'Producto': ['Laptop', 'Laptop', 'Teléfono', 'Teléfono', 'Tablet', 'Tablet'],
    'Ventas': [200, 300, 50, 100, 10, 20],
    'Región': ['Norte', 'Sur', 'Norte', 'Sur', 'Norte', 'Sur']
}
df = pd.DataFrame(data)

print("--- DataFrame Original ---\n")
df

--- DataFrame Original ---



,Producto,Ventas,Región
0,Laptop,200,Norte
1,Laptop,300,Sur
2,Teléfono,50,Norte
3,Teléfono,100,Sur
4,Tablet,10,Norte
5,Tablet,20,Sur


In [17]:
# Filtrar productos con ventas totales > 150
df_filtrado1 = df.groupby('Producto').filter(lambda x: x['Ventas'].sum() > 150)
print("\n--- Productos con ventas totales > 150 ---\n")
df_filtrado1


--- Productos con ventas totales > 150 ---



,Producto,Ventas,Región
0,Laptop,200,Norte
1,Laptop,300,Sur


In [18]:
# Filtrar productos con al menos una venta > 150
df_filtrado2 = df.groupby('Producto').filter(lambda x: x['Ventas'].max() > 150)
print("\n--- Productos con al menos una venta > 150 ---\n")
df_filtrado2


--- Productos con al menos una venta > 150 ---



,Producto,Ventas,Región
0,Laptop,200,Norte
1,Laptop,300,Sur


In [19]:
# Filtrar grupos con más de 1 región
df_filtrado3 = df.groupby('Producto').filter(lambda x: x['Región'].nunique() > 1)
print("\n--- Productos vendidos en más de una región ---\n")
df_filtrado3


--- Productos vendidos en más de una región ---



,Producto,Ventas,Región
0,Laptop,200,Norte
1,Laptop,300,Sur
2,Teléfono,50,Norte
3,Teléfono,100,Sur
4,Tablet,10,Norte
5,Tablet,20,Sur


### 💡 Explicación

- **`filter()`** devuelve filas donde el grupo cumple la condición
- Si un grupo no cumple, **todas sus filas se eliminan**
- La función lambda recibe cada grupo como un DataFrame

**Casos de uso**:
- Eliminar categorías con pocos datos
- Seleccionar solo grupos con valores extremos
- Filtrar outliers a nivel de grupo

---
## 🎯 Ejemplo 6: Uso de `apply()` para operaciones complejas

**Objetivo**: Aplicar funciones personalizadas complejas que devuelven Series o DataFrames.

`apply()` es la herramienta más flexible pero también la más lenta.

In [20]:
import pandas as pd

# DataFrame de ventas
data = {
    'Vendedor': ['Ana', 'Ana', 'Luis', 'Luis', 'María', 'María'],
    'Producto': ['A', 'B', 'A', 'B', 'A', 'B'],
    'Ventas': [100, 150, 200, 50, 80, 120]
}
df = pd.DataFrame(data)

print("--- DataFrame Original ---\n")
df

--- DataFrame Original ---



,Vendedor,Producto,Ventas
0,Ana,A,100
1,Ana,B,150
2,Luis,A,200
3,Luis,B,50
4,María,A,80
5,María,B,120


In [21]:
# Función compleja: calcular varias métricas por vendedor
def metricas_vendedor(grupo):
    return pd.Series({
        'total_ventas': grupo['Ventas'].sum(),
        'venta_promedio': grupo['Ventas'].mean(),
        'productos_vendidos': grupo['Producto'].nunique(),
        'mejor_producto': grupo.loc[grupo['Ventas'].idxmax(), 'Producto'],
        'venta_maxima': grupo['Ventas'].max()
    })


resultado = df.groupby('Vendedor', group_keys=False).apply(metricas_vendedor, include_groups=False)

print("\n--- Métricas por vendedor ---\n")
resultado


--- Métricas por vendedor ---



,total_ventas,venta_promedio,productos_vendidos,mejor_producto,venta_maxima
Vendedor,,,,,
Ana,250,125.0,2,B,150
Luis,250,125.0,2,A,200
María,200,100.0,2,B,120


In [22]:
# Ejemplo 2: Ranking dentro de cada grupo
df['Ranking'] = df.groupby('Vendedor')['Ventas'].rank(ascending=False, method='dense')

print("\n--- DataFrame con ranking por vendedor ---\n")
df.sort_values(['Vendedor', 'Ranking'])


--- DataFrame con ranking por vendedor ---



,Vendedor,Producto,Ventas,Ranking
1,Ana,B,150,1.0
0,Ana,A,100,2.0
2,Luis,A,200,1.0
3,Luis,B,50,2.0
5,María,B,120,1.0
4,María,A,80,2.0


### 💡 Explicación

- **`apply()`** es la más flexible: puede devolver escalares, Series o DataFrames
- Útil cuando necesitas **múltiples cálculos relacionados**
- **Nota de rendimiento**: `apply()` es más lento que métodos específicos como `agg()` o `transform()`
- **`include_groups=False`**: Parámetro nuevo en pandas >= 2.1 que evita warnings al excluir las columnas de agrupación de la operación

**Cuándo usar cada método**:
- Funciones simples → `agg()`
- Mantener forma original → `transform()`
- Operaciones complejas → `apply()`

---
## 🎯 Ejemplo 7: Caso práctico - Análisis de ventas

**Objetivo**: Aplicar todo lo aprendido en un caso realista.

Vamos a analizar datos de ventas por región, trimestre y producto.

In [23]:
import pandas as pd
import numpy as np

# Crear un dataset realista
np.random.seed(42)
n = 100

data = {
    'Fecha': pd.date_range('2024-01-01', periods=n, freq='D'),
    'Región': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], n),
    'Producto': np.random.choice(['Laptop', 'Tablet', 'Teléfono'], n),
    'Ventas': np.random.randint(50, 500, n),
    'Cantidad': np.random.randint(1, 20, n)
}
df = pd.DataFrame(data)

# Añadir columnas derivadas
df['Trimestre'] = df['Fecha'].dt.quarter
df['Mes'] = df['Fecha'].dt.month


print(f"\nTotal de registros: {len(df)}\n")
print("--- Primeras filas del dataset ---\n")
df.head(8)


Total de registros: 100

--- Primeras filas del dataset ---



,Fecha,Región,Producto,Ventas,Cantidad,Trimestre,Mes
0,2024-01-01,Este,Teléfono,333,3,1,1
1,2024-01-02,Oeste,Tablet,77,19,1,1
2,2024-01-03,Norte,Tablet,157,7,1,1
3,2024-01-04,Este,Tablet,93,9,1,1
4,2024-01-05,Este,Tablet,389,1,1,1
5,2024-01-06,Oeste,Tablet,335,8,1,1
6,2024-01-07,Norte,Tablet,495,7,1,1
7,2024-01-08,Norte,Teléfono,380,18,1,1


In [25]:
# Análisis 1: Ventas totales por región y trimestre
ventas_region = df.groupby(['Región', 'Trimestre']).agg({
    'Ventas': 'sum',
    'Cantidad': 'sum'
}).round(2)

print("\n--- Ventas por región y trimestre ---\n")
ventas_region


--- Ventas por región y trimestre ---



Ventas  Cantidad
Región Trimestre                  
Este   1            6792       227
Norte  1            4973       170
       2             395        10
Oeste  1            8150       269
       2             844        33
Sur    1            5788       163
       2            2148        55

In [27]:
# Análisis 2: Producto más vendido por región
mejor_producto = df.groupby(['Región', 'Producto'])['Ventas'].sum().groupby('Región').idxmax()
print("\n--- Producto más vendido por región ---\n")
mejor_producto


--- Producto más vendido por región ---



,Ventas
Región,
Este,"(Este, Teléfono)"
Norte,"(Norte, Tablet)"
Oeste,"(Oeste, Laptop)"
Sur,"(Sur, Laptop)"


In [30]:
# Análisis 3: Añadir porcentaje de ventas respecto al total de la región
df['Total_región'] = df.groupby('Región')['Ventas'].transform('sum')
df['Porcentaje_región'] = (df['Ventas'] / df['Total_región'] * 100).round(2)

print("\n--- Muestra con porcentaje de ventas ---\n")
df[['Fecha', 'Región', 'Producto', 'Ventas', 'Porcentaje_región']].head(8)


--- Muestra con porcentaje de ventas ---



,Fecha,Región,Producto,Ventas,Porcentaje_región
0,2024-01-01,Este,Teléfono,333,4.90
1,2024-01-02,Oeste,Tablet,77,0.86
2,2024-01-03,Norte,Tablet,157,2.92
3,2024-01-04,Este,Tablet,93,1.37
4,2024-01-05,Este,Tablet,389,5.73
5,2024-01-06,Oeste,Tablet,335,3.72
6,2024-01-07,Norte,Tablet,495,9.22
7,2024-01-08,Norte,Teléfono,380,7.08


In [31]:
# Análisis 4: Resumen ejecutivo
resumen = df.groupby('Región').agg({
    'Ventas': ['sum', 'mean', 'std'],
    'Cantidad': 'sum',
    'Producto': lambda x: x.value_counts().index[0]  # Producto más común
}).round(2)

resumen.columns = ['Ventas_total', 'Ventas_promedio', 'Ventas_std', 'Cantidad_total', 'Producto_principal']

print("\n--- Resumen ejecutivo por región ---\n")
resumen


--- Resumen ejecutivo por región ---



,Ventas_total,Ventas_promedio,Ventas_std,Cantidad_total,Producto_principal
Región,,,,,
Este,6792,283.00,124.75,227,Teléfono
Norte,5368,268.40,118.19,180,Tablet
Oeste,8994,299.80,121.96,302,Laptop
Sur,7936,305.23,104.31,218,Teléfono


### 💡 Análisis del caso práctico

Este ejemplo combina varias técnicas:
1. **Agrupación múltiple** con `groupby(['Región', 'Trimestre'])`
2. **Transformación** para añadir columnas calculadas
3. **Agregaciones personalizadas** con funciones lambda
4. **Anidación de groupby** para encontrar máximos por grupo

Es un patrón típico en análisis de negocio.

---
## 🎯 Ejemplo 8: `pivot_table` como alternativa a groupby

**Objetivo**: Conocer `pivot_table`, una alternativa útil para crear tablas de resumen.

`pivot_table` es esencialmente un wrapper conveniente alrededor de `groupby` con reshape automático.

In [33]:
import pandas as pd

# Datos de ventas
data = {
    'Región': ['Norte', 'Sur', 'Norte', 'Sur', 'Este', 'Este'] * 2,
    'Trimestre': ['Q1', 'Q1', 'Q2', 'Q2', 'Q1', 'Q2'] * 2,
    'Producto': ['A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B'],
    'Ventas': [100, 150, 200, 120, 80, 90, 110, 130, 180, 140, 85, 95]
}
df = pd.DataFrame(data)

print("--- DataFrame Original ---\n")
df

--- DataFrame Original ---



,Región,Trimestre,Producto,Ventas
0,Norte,Q1,A,100
1,Sur,Q1,A,150
2,Norte,Q2,A,200
3,Sur,Q2,A,120
4,Este,Q1,A,80
5,Este,Q2,A,90
6,Norte,Q1,B,110
7,Sur,Q1,B,130
8,Norte,Q2,B,180
9,Sur,Q2,B,140


In [34]:
# Usando groupby (más verboso)
resultado_groupby = df.groupby(['Región', 'Trimestre'])['Ventas'].sum().unstack()
print("\n--- Resultado con groupby ---\n")
resultado_groupby


--- Resultado con groupby ---



Trimestre,Q1,Q2
Región,,
Este,165,185
Norte,210,380
Sur,280,260


In [35]:
# Usando pivot_table (más intuitivo para tablas)
resultado_pivot = pd.pivot_table(
    df,
    values='Ventas',
    index='Región',
    columns='Trimestre',
    aggfunc='sum',
    fill_value=0
)
print("\n--- Resultado con pivot_table ---\n")
resultado_pivot


--- Resultado con pivot_table ---



Trimestre,Q1,Q2
Región,,
Este,165,185
Norte,210,380
Sur,280,260


In [36]:
# Pivot table con múltiples agregaciones
pivot_completo = pd.pivot_table(
    df,
    values='Ventas',
    index='Región',
    columns='Producto',
    aggfunc=['sum', 'mean'],
    fill_value=0
)
print("\n--- Pivot table con múltiples agregaciones ---\n")
pivot_completo


--- Pivot table con múltiples agregaciones ---



sum        mean       
Producto    A    B      A      B
Región                          
Este      170  180   85.0   90.0
Norte     300  290  150.0  145.0
Sur       270  270  135.0  135.0

### 💡 Explicación

**¿Cuándo usar pivot_table vs groupby?**

- **`pivot_table`**: Cuando quieres una tabla de resumen tipo Excel (filas × columnas)
- **`groupby`**: Cuando necesitas más control o operaciones complejas

**Ventajas de pivot_table**:
- Más legible para tablas de resumen
- Parámetro `fill_value` para valores faltantes
- Automáticamente añade totales con `margins=True`

---
## ⚠️ Errores comunes y cómo evitarlos

Esta sección cubre los errores más frecuentes al usar `groupby`.

### Error 1: Olvidar reset_index()

In [40]:
import pandas as pd

df = pd.DataFrame({
    'Categoría': ['A', 'B', 'A'],
    'Valor': [10, 20, 30]
})

# ❌ Problema: La categoría queda como índice
resultado_malo = df.groupby('Categoría')['Valor'].sum()

print(f"Tipo: {type(resultado_malo)}")
print(f"¿Es 'Categoría' un Índice? {('Categoría' in resultado_malo.index.names)}\n")

print("--- Sin reset_index() ---\n")
resultado_malo

Tipo: <class 'pandas.core.series.Series'>
¿Es 'Categoría' un Índice? True

--- Sin reset_index() ---



,Valor
Categoría,
A,40
B,20


In [39]:
# ✅ Solución: Usar reset_index()
resultado_bueno = df.groupby('Categoría')['Valor'].sum().reset_index()

print(f"Tipo: {type(resultado_bueno)}")
print(f"¿Es 'Categoría' una columna? {('Categoría' in resultado_bueno.columns)}\n")

print("\n--- Con reset_index() ---\n")
resultado_bueno

Tipo: <class 'pandas.core.frame.DataFrame'>
¿Es 'Categoría' una columna? True


--- Con reset_index() ---



,Categoría,Valor
0,A,40
1,B,20


### Error 2: Confundir agg(), transform() y apply()

In [41]:
import pandas as pd

df = pd.DataFrame({
    'Grupo': ['A', 'A', 'B', 'B'],
    'Valor': [10, 20, 15, 25]
})

print("--- DataFrame Original ---\n")
df

--- DataFrame Original ---



,Grupo,Valor
0,A,10
1,A,20
2,B,15
3,B,25


In [42]:
# agg() reduce los grupos

print(f"Filas resultado: {len(df.groupby('Grupo')['Valor'].agg('mean'))}")

print("\n--- agg(): Reduce grupos ---\n")
df.groupby('Grupo')['Valor'].agg('mean')

Filas resultado: 2

--- agg(): Reduce grupos ---



,Valor
Grupo,
A,15.0
B,20.0


In [43]:
# transform() mantiene el tamaño
resultado_transform = df.groupby('Grupo')['Valor'].transform('mean')
print(f"Filas resultado: {len(resultado_transform)}")

print("\n--- transform(): Mantiene tamaño ---\n")
resultado_transform

Filas resultado: 4

--- transform(): Mantiene tamaño ---



,Valor
0,15.0
1,15.0
2,20.0
3,20.0


In [44]:
# apply() es flexible
print("\n--- apply(): Flexible ---\n")
df.groupby('Grupo')['Valor'].apply(lambda x: x.max() - x.min())


--- apply(): Flexible ---



,Valor
Grupo,
A,10
B,10


### Error 3: No entender el comportamiento de NaN

In [45]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'Grupo': ['A', 'A', 'B', 'B'],
    'Valor': [10, np.nan, 20, 30]
})

print("--- DataFrame con NaN ---")
df

--- DataFrame con NaN ---


,Grupo,Valor
0,A,10.0
1,A,NaN
2,B,20.0
3,B,30.0


In [46]:
# Por defecto, las funciones de agregación ignoran NaN
print("\n--- sum() ignora NaN ---\n")
df.groupby('Grupo')['Valor'].sum()


--- sum() ignora NaN ---



,Valor
Grupo,
A,10.0
B,50.0


In [47]:
# Si quieres contar NaN como 0, rellena primero
print("\n--- sum() después de fillna(0) ---\n")
df.fillna(0).groupby('Grupo')['Valor'].sum()


--- sum() después de fillna(0) ---



,Valor
Grupo,
A,10.0
B,50.0


In [48]:
# count() solo cuenta valores no nulos
print("\n--- count() no cuenta NaN ---\n")
df.groupby('Grupo')['Valor'].count()


--- count() no cuenta NaN ---



,Valor
Grupo,
A,1
B,2


In [49]:
# size() cuenta todas las filas
print("\n--- size() cuenta todas las filas ---\n")
df.groupby('Grupo').size()


--- size() cuenta todas las filas ---



,0
Grupo,
A,2
B,2


### Error 4: Problemas de rendimiento con apply()

In [ ]:
import pandas as pd
import numpy as np
import time

# Crear un DataFrame grande
df = pd.DataFrame({
    'Grupo': np.random.choice(['A', 'B', 'C'], 10000),
    'Valor': np.random.randn(10000)
})

# ❌ Lento: Usar apply() para operaciones simples
inicio = time.time()
resultado_lento = df.groupby('Grupo')['Valor'].apply(lambda x: x.mean())
tiempo_lento = time.time() - inicio

# ✅ Rápido: Usar funciones optimizadas
inicio = time.time()
resultado_rapido = df.groupby('Grupo')['Valor'].mean()
tiempo_rapido = time.time() - inicio

print(f"Tiempo con apply(): {tiempo_lento:.4f} segundos")
print(f"Tiempo con mean(): {tiempo_rapido:.4f} segundos")
print(f"\nMejora de velocidad: {tiempo_lento/tiempo_rapido:.1f}x más rápido")
print("\n⚠️ Lección: Usa métodos específicos (mean, sum, etc.) en lugar de apply() cuando sea posible")

Tiempo con apply(): 0.0072 segundos
Tiempo con mean(): 0.0022 segundos

Mejora de velocidad: 3.3x más rápido

⚠️ Lección: Usa métodos específicos (mean, sum, etc.) en lugar de apply() cuando sea posible


---
## 📋 Resumen y mejores prácticas

### Cuadro resumen de métodos

| Método | ¿Qué devuelve? | Cuándo usarlo |
|--------|---------------|---------------|
| `agg()` | Reduce grupos | Estadísticas agregadas (suma, media, etc.) |
| `transform()` | Mantiene tamaño | Añadir columnas calculadas por grupo |
| `apply()` | Flexible | Operaciones complejas personalizadas |
| `filter()` | Filtra grupos completos | Mantener/eliminar grupos según condición |
| `size()` | Cuenta filas | Número de elementos por grupo (incluye NaN) |
| `count()` | Cuenta no-nulos | Número de valores válidos por columna |

### Mejores prácticas

1. **Rendimiento**:
   - Usa funciones específicas (`mean`, `sum`) en lugar de `apply()` cuando sea posible
   - Para DataFrames grandes, considera `category` dtype para columnas de agrupación

2. **Legibilidad**:
   - Usa `reset_index()` para convertir índices en columnas
   - Nombra tus agregaciones: `agg(total=('Ventas', 'sum'))`
   - Considera `pivot_table` para tablas de resumen

3. **Manejo de datos**:
   - Sé consciente de cómo se manejan los NaN
   - Usa `dropna=False` en groupby para incluir grupos con NaN
   - Valida que los grupos resultantes son los esperados

4. **Debugging**:
   - Usa `df.groupby('col').size()` para ver el tamaño de cada grupo
   - Imprime grupos individuales con `df.groupby('col').get_group('valor')`
   - Verifica el tipo del resultado (`Series` vs `DataFrame`)

### Recursos adicionales

- Documentación oficial: https://pandas.pydata.org/docs/user_guide/groupby.html
- Para casos más avanzados, explora: `grouper()`, `rolling()`, `expanding()`

---
## 🎓 Ejercicios propuestos

Para practicar lo aprendido, intenta resolver estos ejercicios:

1. **Básico**: Crea un DataFrame de estudiantes con notas y calcula la media por clase.

2. **Intermedio**: Dado un DataFrame de transacciones, encuentra el cliente que más gastó cada mes.

3. **Avanzado**: Normaliza las ventas de cada producto por región (media=0, std=1) usando `transform()`.

4. **Desafío**: Crea una función que identifique outliers por grupo (valores > 2 desviaciones estándar) y márcalos en una nueva columna.